# Adversarial Chess
6.4212 Final Project

Kameron Dawson

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (
    DiagramBuilder, StartMeshcat, Simulator, Rgba, RigidTransform
)
from pydrake.geometry import Sphere
from manipulation.station import LoadScenario, MakeHardwareStation, AddPointClouds
from manipulation.utils import RenderDiagram
from setup_simulation import get_scenario
from utils import visualize_box, visualize_text
from perception.point_cloud import get_scene_point_cloud, get_model_point_clouds
from perception.clustering import cluster_point_cloud_kmeans, cluster_point_cloud_dbscan, cluster_point_cloud_hybrid
from perception.perception import segment_scene_point_cloud, classify_piece_colors, match_scene_to_model_cloud_icp, match_scene_to_model_cloud_bb

In [ ]:
# Setup meshcat for visualization
meshcat = StartMeshcat()
print('Click the link above to open Meshcat in your browser!')

In [ ]:
# Clear meshcat
meshcat.Delete()

In [ ]:
# Get scenario
scenario_string = get_scenario()

# Load the scenario and build the simulation station
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)

# Build a Drake Diagram containing the station
builder = DiagramBuilder()
builder.AddSystem(station)

# Setup getting point clouds from the cameras
to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(to_point_cloud["camera0"].get_output_port(), "camera0_point_cloud")
builder.ExportOutput(to_point_cloud["camera1"].get_output_port(), "camera1_point_cloud")
builder.ExportOutput(to_point_cloud["camera2"].get_output_port(), "camera2_point_cloud")

# Build diagram
diagram = builder.Build()

In [ ]:
RenderDiagram(diagram, max_depth=1)

In [ ]:
# Get relevant vars from the diagram
context = diagram.CreateDefaultContext()

In [ ]:
# Create and run a simulator
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(0.1)

## Perception

### Sanity Checks

In [ ]:
# Run the following cell to visualize the rgb outputs of each of the cameras
cameras = ["camera0", "camera1", "camera2"]
station_context = diagram.GetSubsystemContext(station, context)

fig, axes = plt.subplots(
    1, len(cameras), figsize=(5 * len(cameras), 4), constrained_layout=True
)
for ax, cam in zip(axes, cameras):
    img = station.GetOutputPort(f"{cam}.rgb_image").Eval(station_context)
    arr = np.array(img.data, copy=False).reshape(img.height(), img.width(), -1)
    im = ax.imshow(arr)
    ax.set_title(f"{cam} rgb image")
    ax.axis("off")

plt.show()

In [ ]:
# Run the following cell to visualize the depth outputs of each of the cameras
fig, axes = plt.subplots(
    1, len(cameras), figsize=(5 * len(cameras), 4), constrained_layout=True
)
for ax, cam in zip(axes, cameras):
    img = station.GetOutputPort(f"{cam}.depth_image").Eval(station_context)
    depth_img = np.array(img.data, copy=False).reshape(img.height(), img.width(), -1)
    depth_img = np.ma.masked_invalid(depth_img)
    img = ax.imshow(depth_img, cmap="magma")
    ax.set_title(f"{cam} depth image")
    ax.axis("off")

plt.show()

### ICP

In [ ]:
# Get scene point cloud from depth cameras, then segment it by cropping out known objects (floor, table, chessboard, iiwas)
scene_point_cloud = segment_scene_point_cloud(get_scene_point_cloud(diagram, context))

In [ ]:
# Sanity checks of scene point cloud
print('Num scene points: ', scene_point_cloud.size())
xyzs = scene_point_cloud.xyzs()
x = xyzs[0]
y = xyzs[1]
z = xyzs[2]

print('min, max xyz:')
min_x, max_x = min(x), max(x)
min_y, max_y = min(y), max(y)
min_z, max_z = min(z), max(z)
print(min_x, max_x)
print(min_y, max_y)
print(min_z, max_z)

In [ ]:
# Visualize scene point cloud
meshcat.SetObject(
    "scene_point_cloud", scene_point_cloud, point_size=0.05, rgba=Rgba(1, 0, 0)
)

In [ ]:
# Scene point cloud only contains piece clouds
# piece_clouds = cluster_point_cloud_kmeans(scene_point_cloud)
# piece_clouds = cluster_point_cloud_dbscan(scene_point_cloud)
piece_clouds = cluster_point_cloud_hybrid(scene_point_cloud)

In [ ]:
# Visual piece cloud centroids
for i, piece_cloud in enumerate(piece_clouds):
    pts = piece_cloud.xyzs()
    centroid = np.mean(pts, axis=1)
    X = RigidTransform()
    X.set_translation(centroid)
    path = f'/piece/{i}'
    meshcat.SetTransform(path, X)
    meshcat.SetObject(
        path, Sphere(0.01), Rgba(0, 0, 1)
    )

In [ ]:
# Visualize piece clouds
for i, cluster in enumerate(piece_clouds):
    meshcat.SetObject(
        f"cluster_point_cloud_{i}", cluster, point_size=0.05, rgba=Rgba(1, 0, 0)
    )

In [ ]:
# Separate piece clouds into the dark and light sets
color_classifications = classify_piece_colors(piece_clouds)
piece_clouds_by_color = {'dark': [], 'light': []}
for color, cluster in zip(color_classifications, piece_clouds):
    piece_clouds_by_color[color].append(cluster)
    

In [ ]:
# Visualize color classified piece clouds
i = 0
for color, cluster in zip(color_classifications, piece_clouds):
    rgb = Rgba(1, 1, 1) if color == 'light' else Rgba(0, 0, 0)
    meshcat.SetObject(
        f"cluster_point_cloud_{color}_{i}", cluster, point_size=0.0025, rgba=rgb
    )
    i += 1

In [ ]:
# Get model point clouds
model_point_clouds = get_model_point_clouds(n_sample_points=100)

In [ ]:
# Match scene clusters to model clouds using bounding boxes
dark_poses = match_scene_to_model_cloud_bb(piece_clouds_by_color['dark'], model_point_clouds['pieces']['dark'])
light_poses = match_scene_to_model_cloud_bb(piece_clouds_by_color['light'], model_point_clouds['pieces']['light'])

In [ ]:
# Match scene clusters to model clouds
dark_poses = match_scene_to_model_cloud_icp(piece_clouds_by_color['dark'], model_point_clouds['pieces']['dark'])
light_poses = match_scene_to_model_cloud_icp(piece_clouds_by_color['light'], model_point_clouds['pieces']['light'])

In [ ]:
# Visualize poses found by ICP
piece_counts = {
    'pawn': 8,
    'rook': 2,
    'knight': 2,
    'bishop': 2,
    'queen': 1,
    'king': 1,
}

# Dark pieces
for piece in piece_counts:
    for k in range(1, piece_counts[piece] + 1):
        name = f'{piece}{k}'
        X = dark_poses[name]
        path = f'/dark/{name}'
        meshcat.SetTransform(path, X)
        meshcat.SetObject(
            path, Sphere(0.01), Rgba(1, 0, 0)
        )

# Light pieces
for piece in piece_counts:
    for k in range(1, piece_counts[piece] + 1):
        name = f'{piece}{k}'
        X = light_poses[name]
        path = f'/light/{name}'
        meshcat.SetTransform(path, X)
        meshcat.SetObject(
            path, Sphere(0.01), Rgba(0, 1, 0)
        )

In [ ]:
colors = {
    'pawn': Rgba(1, 0, 0), # red
    'king': Rgba(0, 1, 0), # green
    'queen': Rgba(0, 0, 1), # blue
    'bishop': Rgba(1, 1, 0), # yellow
    'knight': Rgba(0, 1, 1), # cyan
    'rook': Rgba(1, 0, 1) # magenta
}

for piece, cloud in zip(dark_poses, piece_clouds_by_color['dark']):
    name = piece[:-1]
    # Get piece pose
    pts = cloud.xyzs()
    centroid = np.mean(pts, axis=1)
    X = RigidTransform()
    X.set_translation(centroid)
    # Draw piece name
    # pose = RigidTransform()
    # pose.set_translation([centroid[0], centroid[1], 0.6])
    # visualize_text(meshcat, name, X, piece)
    # Visualize centroid
    path = f'/piece/{piece}'
    meshcat.SetTransform(path, X)
    meshcat.SetObject(
        path, Sphere(0.01), rgba=colors[name]
    )

In [ ]:
for cloud in piece_clouds_by_color['dark']:
    pts = cloud.xyzs()
    x, y, z = pts[0], pts[1], pts[2]
    min_x, max_x = np.min(x), np.max(x)
    min_y, max_y = np.min(y), np.max(y)
    min_z, max_z = np.min(z), np.max(z)
    lower, upper = np.array([min_x, min_y, min_z]), np.array([max_x, max_y, max_z])
    visualize_box(meshcat, lower, upper)